# 150. PyTorch 다중 분류 손실 함수

## Categorical Crossentropy (범주형 교차 엔트로피)
    
<img src="https://gombru.github.io/assets/cross_entropy_loss/softmax_CE_pipeline.png" width=500 />

### 🔹 `nn.CrossEntropyLoss` 란?
- **PyTorch에서 다중 클래스 분류 문제**를 해결할 때 사용하는 손실 함수입니다.
- `nn.LogSoftmax()`와 `nn.NLLLoss()`를 **하나의 클래스로 결합**한 형태입니다.

> 즉,  
> `nn.CrossEntropyLoss()` = `nn.NLLLoss(nn.LogSoftmax())`

---

## 주요 수식 및 개념

### 1️⃣ **Cross Entropy Loss (교차 엔트로피 손실)**
교차 엔트로피 손실은 모델이 예측한 확률 분포와 실제 정답 사이의 차이를 측정하는 방법입니다.

$$
loss(x, class) = -\log(\frac{\exp(x[class])}{\sum_{j}{\exp(x[j])}})
$$

위 식을 풀어서 보면:

- (x[class]) : 정답 클래스의 출력 값 (logit)
- $\sum_{j}{\exp(x[j])}$ : 모든 클래스에 대해 `softmax`를 적용한 분모 (정규화)
- $-x[class] + \log(\sum_{j}\exp(x[j]))$ : 최종적으로 음의 로그 가능도를 계산

---

### 2️⃣ **LogSoftmax (로그 소프트맥스)**
`softmax` 함수에 로그를 적용한 형태입니다.

$$
LogSoftmax(x_i) = \log(\frac{\exp(x_i)}{\sum_{j}{\exp(x_j)}})
$$

- 왜 LogSoftmax를 사용할까?
  - 수학적으로 안정성이 높아 소수점 연산 오류(Underflow)를 방지할 수 있음.
  - Softmax 후 로그를 따로 취하는 것보다 더 안전한 방식.

---

### 3️⃣ **NLLLoss (Negative Log Likelihood Loss, 음의 로그 우도 손실)**
`nn.NLLLoss()`는 **확률 값 대신 로그 확률(log probability)을 입력으로 받는 손실 함수**입니다.

$$
l(x, y) = - \frac{1}{N} \sum_{1}^{N}l_n
$$

- `CrossEntropyLoss`는 내부적으로 `LogSoftmax`를 포함하고 있으므로,  
  **입력 값이 logits일 때 바로 사용 가능**합니다.
- `NLLLoss`는 로그 확률 값을 입력으로 받으므로,  
  **사용할 때 `LogSoftmax`를 별도로 적용해야 함**.

---

## 결론: `nn.CrossEntropyLoss()` vs `nn.NLLLoss()`
| 손실 함수 | 입력 데이터 | 내부 처리 과정 |
|-----------|------------|----------------|
| **`nn.CrossEntropyLoss()`** | logits (정규화 X) | LogSoftmax + NLLLoss 포함 |
| **`nn.NLLLoss()`** | log-probabilities (로그 확률) | LogSoftmax를 먼저 적용해야 함 |

> **요약:**  
> - `nn.CrossEntropyLoss()`를 사용할 경우, **모델의 마지막 레이어에서 Softmax를 적용하지 않아야 함!**  
> - `nn.NLLLoss()`를 사용할 경우, **모델의 마지막 레이어에서 LogSoftmax를 적용해야 함!**

### 결론적으로, Pytorch 에서는 Cross Entropy Loss 값의 결과를 얻기 위해 2가지 방식이 존재

### 방법 1
### nn.CrossEntropyLoss 함수 사용

-  `NNLLoss(Log(Softmax))`

$-x[class] + log(\sum_{j}exp(x[j]))$

In [ ]:
# 이 텐서는 모델의 출력으로 볼 수 있으며, 각 요소는 특정 클래스에 속할 확률 또는 점수를 나타냅니다.
# 여기서는 10개의 다른 클래스에 대한 점수를 포함하고 있습니다.
# 주어진 샘플의 실제 클래스를 1번 클래스로 지정

In [ ]:
# 크로스 엔트로피 손실 함수 생성 (Softmax + NLLLoss 포함)
# 예측값 x와 실제 레이블 y에 대해 손실 계산
# 손실값 출력

### 방법 2
### nn.LogSoftmax + nn.NLLLoss

- LogSoftmax : Log + Softmax - 두 함수를 따로 적용한 것 보다 수학적 안정성이 좋다.

In [ ]:
# LogSoftmax 함수 생성 (소프트맥스 적용 후 로그 값 반환)
# 입력 x에 대해 로그 소프트맥스 계산 (클래스별 로그 확률)

In [ ]:
# torch.nn.NLLLoss()를 인스턴스화하고, 계산된 로그 소프트맥스 값(x_log)과
# 실제 레이블(y)을 사용하여 음의 로그 가능도 손실(Negative Log Likelihood Loss)을 계산합니다.

### 사용상의 주의 사항

#### nn.CrossEntropyLoss 를 사용할 경우
-  nn.CrossEntropyLoss 내에 softmax 함수가 포함되어 있으므로 Neural Network의 마지막 activation 함수로 Softmax를 지정 않고, logit 만 출력한다.  

#### nn.NLLLoss 를 사용할 경우
- nn.NLLLoss 는 입력으로 확률 분포가 와야 하므로, Neural Network의 마지막 actiovation 함수로 LogSoftmax를 지정 한다.

### forward method 의 return value

In [ ]:
# nn.Module을 상속받아 사용자 정의 신경망 클래스를 정의합니다.
class Ex_NN(nn.Module):
    def __init__(self):
        # 첫 번째 선형 레이어를 정의합니다. 입력 차원은 2, 출력 차원은 8입니다.
        # 두 번째 선형 레이어를 정의합니다. 입력 차원은 8, 출력 차원은 3입니다.
        # LogSoftmax 활성화 함수를 정의합니다.
    def forward1(self, x):
        # CrossEntropyLoss를 사용할 경우, 마지막에 log softmax를 사용하지 않습니다.
    def forward2(self, x):
        # NLLLoss를 사용할 경우, 마지막에 log softmax를 적용합니다.

In [ ]:
# Ex_NN 모델 인스턴스 생성
# 입력 데이터 (3개의 샘플, 각 샘플은 2개의 특성값)
# 정답 레이블 (각 샘플의 클래스 인덱스)

- CrossEntropyLoss 사용

- Negative Log Likelihood Loss 사용

### category 분류

In [ ]:
# prob는 모델의 출력 확률 분포를 나타내는 텐서입니다.
# torch.argmax 함수는 주어진 차원(axis)에 대해 최대값을 갖는 인덱스를 반환합니다.
# 여기서 axis=-1은 텐서의 마지막 차원을 의미합니다. 다중 클래스 분류 문제에서는
# 각 샘플의 예측된 클래스 인덱스를 찾는 데 사용됩니다.

In [ ]:
# `torch.max` 함수는 주어진 텐서에서 최대 값을 찾습니다.
# 이 함수는 두 개의 반환값을 가집니다: 최대 값과 해당 값의 인덱스입니다.
# `prob` 변수는 특정 확률 분포 또는 점수가 포함된 텐서를 가정합니다.
# `axis=-1` 매개변수는 최대 값을 찾을 차원을 지정합니다.
# `-1`은 텐서의 마지막 차원을 의미합니다. 이는 다차원 텐서에서 각 벡터별로 최대 값을 찾는 데 사용됩니다.